In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler



%matplotlib inline

In [ ]:

train=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")

test=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")

sub=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")


In [ ]:
# EDA
display(train.head())
display(test.head())
display(sub.head())

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
feature_columns= train.drop(['id', 'target'], axis=1).columns
feature_columns

In [ ]:
train_con = train[feature_columns].select_dtypes(include=[np.number])
train_cat = train[feature_columns].select_dtypes(exclude=[np.number])
display(train_con.head())
display(train_cat.head())

In [ ]:
train_ind = train.index
test_ind = test.index

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,8))

sns.kdeplot(data=train, x="target",ax=ax[0])
sns.boxplot(x='target', data=train, orient='h', ax=ax[1]);

In [ ]:
f, axes = plt.subplots(4,4, figsize=(15, 15))
f.suptitle('Features Distribution', fontsize=14)

for index, column in enumerate(train_con.columns):
    i,j = (index // 4, index % 4)
    g = sns.kdeplot(train_con[column], color="b", shade=True, label="%.2f"%(train_con[column].skew()), ax=axes[i,j])
    g = g.legend(loc="best")

In [ ]:
f,ax=plt.subplots(figsize=(14,8))

sns.heatmap(train_con.corr(),annot=True,ax=ax)

In [ ]:
f,ax=plt.subplots(4,3,figsize=(16,16))
for index, column in enumerate(train_cat.columns):
    i,j = (index // 4, index % 4)
    sns.countplot(x=column, data=train_cat,ax=ax[j,i])

In [ ]:
for feature in train_cat.columns:
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])
display(train.head())
display(test.head())

In [ ]:
# Base Model
x = train[feature_columns]
y = train[['target']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
scaler_x = MinMaxScaler().fit(x_train)
scaler_y = MinMaxScaler().fit(y_train)

In [ ]:
x_train_sc = scaler_x.transform(x_train)
x_test_sc = scaler_x.transform(x_test)
y_train_sc = scaler_y.transform(y_train)
y_test_sc = scaler_y.transform(y_test)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train_sc, y_train_sc)
y_pred_sc = lr_model.predict(x_test_sc)

In [ ]:
mae = mean_absolute_error(y_test_sc, y_pred_sc)
rmse = np.sqrt(mean_squared_error(y_test_sc, y_pred_sc))

print('MAE = ', mae.round(4))
print('RMSE = ', rmse.round(4))

In [ ]:
y_test_inv = scaler_y.inverse_transform(y_test_sc.reshape(-1,1))
y_pred_inv = scaler_y.inverse_transform(y_pred_sc.reshape(-1,1))

actual_mae = mean_absolute_error(y_test_inv, y_pred_inv)
actual_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

print('Actual MAE = ', actual_mae)
print('Actual RMSE = ', actual_rmse)

In [ ]:
test_actual = test.drop('id', axis=1)
test_actul_sc = scaler_x.transform(test_actual)
test_actual_scaler_y_sc = lr_model.predict(test_actul_sc)

In [ ]:
Resalut = pd.DataFrame()
Resalut['id']=test.id
Resalut['target']=test_actual_scaler_y_sc
Resalut.to_csv('submission.csv')
Resalut.set_index('id').to_csv('submission.csv')